In [35]:
import rdkit

In [36]:
import rdkit
from rdkit import Chem
from rdkit.Chem import Draw
import os

In [37]:
import tensorflow as tf

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import numpy as np
import pandas as pd
import time
from tqdm import tqdm

import warnings
warnings.filterwarnings(action = 'ignore')

분자구조 이미지 SMILES 변환 BASELINE

In [38]:
PATH = pd.read_csv('./smiles data/train.csv')

In [39]:
PATH.head()

,file_name,SMILES
0,train_0.png,CC(=O)OC(CC(=O)[O-])C[N+](C)(C)C
1,train_1.png,CC(=O)OC(CC(=O)O)C[N+](C)(C)C
2,train_2.png,O=C(O)C1=CC=CC(O)C1O
3,train_3.png,CC(O)CN
4,train_4.png,NCC(=O)COP(=O)(O)O


ChemHub에서 수집한 데이터를 이용해 학습
제공 된 데이터는 ChemHub에서 순서대로 수집한 SMILES로 유사 구조들이 밀집되어 있을 가능성이 높습니다.

In [1]:
with open('./smiles data/train.csv', 'r') as csv_file:
    data = csv_file.read()
    
    
all_captions = []
all_img_name_vector = []

for line in data.split('\n')[1:-1]:
    image_id, smiles = line.split(',')   
    caption = '<' + smiles + '>'
    full_image_path = PATH + image_id

    all_img_name_vector.append(full_image_path)
    all_captions.append(caption)

train_captions, img_name_vector = shuffle(all_captions, all_img_name_vector, random_state=42)

num_examples = 908765 # 학습에 사용할 데이터 수, Baseline에서는 제공된 데이터 모두 사용하였습니다.
train_captions = train_captions[:num_examples]
img_name_vector = img_name_vector[:num_examples]

NameError: name 'PATH' is not defined

입력에 맞게 이미지를 로드하는 함수 정의

In [ ]:
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img, image_path

imagenet으로 pretrain된 InceptionV3를 이용하여 이미지의 feature 추출

In [ ]:
image_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')
new_input = image_model.input
hidden_layer = image_model.layers[-1].output

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

빠른 학습을 위해 추출된 feature 로컬디스크에 저장

In [ ]:
encode_train = sorted(set(img_name_vector))

image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)
image_dataset = image_dataset.map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(16)

for img, path in tqdm(image_dataset):
    batch_features = image_features_extract_model(img)
    batch_features = tf.reshape(batch_features, (batch_features.shape[0], -1, batch_features.shape[3]))

    for bf, p in zip(batch_features, path):
        path_of_feature = p.numpy().decode("utf-8")
        np.save(path_of_feature, bf.numpy())

예측할 SMILES의 최대 길이 정의

학습에 사용하는 SMILES의 길이는 70이하로 샘플링된 상태

TEST SET은 70이상의 SMILES도 존재합니다.

In [19]:
#
def calc_max_length(tensor):
    return max(len(t) for t in tensor)

In [20]:
max_length = calc_max_length(train_captions)

NameError: name 'train_captions' is not defined

SMILES 토큰화

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(lower=False, char_level=True)
tokenizer.fit_on_texts(train_captions)
top_k = len(tokenizer.word_index)

In [ ]:
train_seqs = tokenizer.texts_to_sequences(train_captions)

In [ ]:
cap_vector = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post')

학습, 검증 데이터셋 분리

In [ ]:
img_name_train, img_name_val, cap_train, cap_val = train_test_split(img_name_vector, cap_vector, test_size=0.02, random_state=42)

In [ ]:
len(img_name_train), len(cap_train), len(img_name_val), len(cap_val)